In [1]:
## 02.
# 고객 891명에 대한 학습용 데이터를 이용하여 생존 여부를 예측하는 모형을 만듭니다.
# 이후 예측 모형을 평가용 데이터에 적용하여 418명 승객의 생존여부 예측값을 다음과 같은 형식의 csv 파일로 생성하시오.
# 제출한 모델의 성능은 ROC-AUC 평가지표에 따라 채점
# 성능이 우수한 예측 모형을 구축하려면 적절한 데이터 전처리, Feature Engineering, 분류 알고리즘 사용, 하이퍼 매개변수 최적화, 모형 앙상블 등이 수반되어야 한다.

In [9]:
import pandas as pd
x_train = pd.read_csv('/Users/dabeen/Desktop/DA_01/bigData-main/titanic_x_train.csv', encoding='cp949')
x_test = pd.read_csv('/Users/dabeen/Desktop/DA_01/bigData-main/titanic_x_test.csv', encoding='cp949')
y_train = pd.read_csv('/Users/dabeen/Desktop/DA_01/bigData-main/titanic_y_train.csv')

print(x_train.info())
print(y_train.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   티켓등급         891 non-null    int64  
 2   승객이름         891 non-null    object 
 3   성별           891 non-null    object 
 4   나이           714 non-null    float64
 5   형제자매배우자수     891 non-null    int64  
 6   부모자식수        891 non-null    int64  
 7   티켓번호         891 non-null    object 
 8   운임요금         891 non-null    float64
 9   객실번호         204 non-null    object 
 10  선착장          889 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 76.7+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   PassengerId  891 non-null    int64
 1   Survived     891 non-null    int64
dtypes: int64(2)


In [13]:
data = pd.concat([x_train, y_train], axis=1)
data

,PassengerId,티켓등급,승객이름,성별,나이,형제자매배우자수,부모자식수,티켓번호,운임요금,객실번호,선착장,PassengerId,Survived
0,1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,1,0
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,2,1
2,3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,3,1
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,4,1
4,5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,5,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S,887,0
887,888,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S,888,1
888,889,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S,889,0
889,890,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C,890,1


In [15]:
print(data.groupby(['성별'])['Survived'].mean())
print(data.groupby(['티켓등급'])['Survived'].mean())
print(data.groupby(['선착장'])['Survived'].mean())

성별
female    0.742038
male      0.188908
Name: Survived, dtype: float64
티켓등급
1    0.629630
2    0.472826
3    0.242363
Name: Survived, dtype: float64
선착장
C    0.553571
Q    0.389610
S    0.336957
Name: Survived, dtype: float64


In [17]:
x_test_passenger_id = x_test['PassengerId']
x_train = x_train.drop(columns=['PassengerId'])
x_test = x_test.drop(columns=['PassengerId'])
y_train = y_train.drop(columns=['PassengerId'])
print(x_train.head(3))
print(x_test.head(3))
print(y_train.head(3))

   티켓등급                                               승객이름      성별    나이  \
0     3                            Braund, Mr. Owen Harris    male  22.0   
1     1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0   
2     3                             Heikkinen, Miss. Laina  female  26.0   

   형제자매배우자수  부모자식수              티켓번호     운임요금 객실번호 선착장  
0         1      0         A/5 21171   7.2500  NaN   S  
1         1      0          PC 17599  71.2833  C85   C  
2         0      0  STON/O2. 3101282   7.9250  NaN   S  
   티켓등급                              승객이름      성별    나이  형제자매배우자수  부모자식수  \
0     3                  Kelly, Mr. James    male  34.5         0      0   
1     3  Wilkes, Mrs. James (Ellen Needs)  female  47.0         1      0   
2     2         Myles, Mr. Thomas Francis    male  62.0         0      0   

     티켓번호    운임요금 객실번호 선착장  
0  330911  7.8292  NaN   Q  
1  363272  7.0000  NaN   S  
2  240276  9.6875  NaN   Q  
   Survived
0         0
1         1
2         

In [18]:
x_train = x_train.drop(columns=['티켓번호', '승객이름'])
x_test = x_test.drop(columns=['티켓번호', '승객이름'])

In [20]:
print(data[['나이', 'Survived']].corr())

                나이  Survived
나이        1.000000 -0.077221
Survived -0.077221  1.000000
          Survived
Survived       1.0
          Survived
Survived       1.0


In [21]:
x_train = x_train.drop(columns=['나이', '객실번호'])
x_test = x_test.drop(columns=['나이', '객실번호'])

In [22]:
print(x_train.groupby(['선착장'])['선착장'].count())

선착장
C    168
Q     77
S    644
Name: 선착장, dtype: int64


In [24]:
x_train['선착장'] = x_train['선착장'].fillna('S')
print(x_train['선착장'].isnull().sum())
print(x_test['선착장'].isnull().sum())


0
0


In [25]:
x_train['성별'] = x_train['성별'].replace('male', 0).replace('female', 1)
x_test['성별'] = x_test['성별'].replace('male', 0).replace('female', 1)

In [26]:
선착장_dummy = pd.get_dummies(x_train['선착장'], drop_first=True).rename(columns={'Q':'선착장Q', 'S':'선착장S'})
x_train = pd.concat([x_train, 선착장_dummy], axis=1)
print(x_train.head())

   티켓등급  성별  형제자매배우자수  부모자식수     운임요금 선착장  선착장Q  선착장S
0     3   0         1      0   7.2500   S     0     1
1     1   1         1      0  71.2833   C     0     0
2     3   1         0      0   7.9250   S     0     1
3     1   1         1      0  53.1000   S     0     1
4     3   0         0      0   8.0500   S     0     1


In [27]:
x_train = x_train.drop(columns=['선착장'])
x_train.head()

,티켓등급,성별,형제자매배우자수,부모자식수,운임요금,선착장Q,선착장S
0,3,0,1,0,7.2500,0,1
1,1,1,1,0,71.2833,0,0
2,3,1,0,0,7.9250,0,1
3,1,1,1,0,53.1000,0,1
4,3,0,0,0,8.0500,0,1


In [28]:
선착장_dummy = pd.get_dummies(x_test['선착장'], drop_first=True).rename(columns={'Q':'선착장Q', 'S':'선착장S'})
x_test = pd.concat([x_test, 선착장_dummy], axis=1)
x_test = x_test.drop(columns=['선착장'])
print(x_train.head())

   티켓등급  성별  형제자매배우자수  부모자식수     운임요금  선착장Q  선착장S
0     3   0         1      0   7.2500     0     1
1     1   1         1      0  71.2833     0     0
2     3   1         0      0   7.9250     0     1
3     1   1         1      0  53.1000     0     1
4     3   0         0      0   8.0500     0     1


In [29]:
x_train['가족수'] = x_train['형제자매배우자수'] + x_train['부모자식수']
x_train = x_train.drop(columns=['형제자매배우자수', '부모자식수'])
x_test['가족수'] = x_test['형제자매배우자수'] + x_test['부모자식수']
x_test = x_test.drop(columns=['형제자매배우자수', '부모자식수'])

print(x_train.head(3))
print(x_test.head(3))

   티켓등급  성별     운임요금  선착장Q  선착장S  가족수
0     3   0   7.2500     0     1    1
1     1   1  71.2833     0     0    1
2     3   1   7.9250     0     1    0
   티켓등급  성별    운임요금  선착장Q  선착장S  가족수
0     3   0  7.8292     1     0    0
1     3   1  7.0000     0     1    1
2     2   0  9.6875     1     0    0


In [30]:
from sklearn.model_selection import train_test_split

X_TRAIN, X_TEST, Y_TRAIN, Y_TEST = train_test_split(x_train, y_train, test_size=0.3, random_state=10)

In [31]:
from xgboost import XGBClassifier
model = XGBClassifier(n_estimators=100, max_depth=5, eval_metric='error', random_state=10)
model.fit(X_TRAIN, Y_TRAIN)

/Users/dabeen/opt/anaconda3/lib/python3.8/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Users/dabeen/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, eval_metric='error',
              gamma=0, gpu_id=-1, importance_type='gain',
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=5, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=8,
              num_parallel_tree=1, random_state=10, reg_alpha=0, reg_lambda=1,
              scale_pos_weight=1, subsample=1, tree_method='exact',
              validate_parameters=1, verbosity=None)

In [32]:
y_test_predicted = pd.DataFrame(model.predict(x_test)).rename(columns={0:'Survived'})
print(pd.DataFrame(y_test_predicted).head(10))

   Survived
0         0
1         1
2         0
3         0
4         1
5         0
6         1
7         0
8         1
9         0


In [33]:
Y_TEST_PREDICTED = pd.DataFrame(model.predict(X_TEST))

In [34]:
from sklearn.metrics import roc_auc_score
print(roc_auc_score(Y_TEST, Y_TEST_PREDICTED))

0.7812423575446319


In [ ]:
final = pd.concat([x_test_passenger_id, Y_TEST_PREDICTED], axis=1)
final.to_csv('/12345.csv', index=False)

In [43]:
import pandas as pd
x_train = pd.read_csv('/Users/dabeen/Desktop/DA_01/bigData-main/titanic_x_train.csv', encoding='cp949')
x_test = pd.read_csv('/Users/dabeen/Desktop/DA_01/bigData-main/titanic_x_test.csv', encoding='cp949')
y_train = pd.read_csv('/Users/dabeen/Desktop/DA_01/bigData-main/titanic_y_train.csv')

data = pd.concat([x_train, y_train], axis=1)

x_test_passenger_id = x_test['PassengerId']
x_train = x_train.drop(columns=['PassengerId'])
x_test = x_test.drop(columns=['PassengerId'])
y_train = y_train.drop(columns=['PassengerId'])
x_train = x_train.drop(columns=['티켓번호', '승객이름'])
x_test = x_test.drop(columns=['티켓번호', '승객이름'])
x_train = x_train.drop(columns=['나이', '객실번호'])
x_test = x_test.drop(columns=['나이', '객실번호'])

x_train['선착장'] = x_train['선착장'].fillna('S')
x_train['성별'] = x_train['성별'].replace('male', 0).replace('female', 1)
x_test['성별'] = x_test['성별'].replace('male', 0).replace('female', 1)

선착장_dummy = pd.get_dummies(x_train['선착장'], drop_first=True).rename(columns={'Q':'선착장Q', 'S':'선착장S'})
x_train = pd.concat([x_train, 선착장_dummy], axis=1)
x_train = x_train.drop(columns=['선착장'])
선착장_dummy = pd.get_dummies(x_test['선착장'], drop_first=True).rename(columns={'Q':'선착장Q', 'S':'선착장S'})
x_test = pd.concat([x_test, 선착장_dummy], axis=1)
x_test = x_test.drop(columns=['선착장'])

x_train['가족수'] = x_train['형제자매배우자수'] + x_train['부모자식수']
x_train = x_train.drop(columns=['형제자매배우자수', '부모자식수'])
x_test['가족수'] = x_test['형제자매배우자수'] + x_test['부모자식수']
x_test = x_test.drop(columns=['형제자매배우자수', '부모자식수'])

from sklearn.model_selection import train_test_split
X_TRAIN, X_TEST, Y_TRAIN, Y_TEST = train_test_split(x_train, y_train, test_size=0.3, random_state=10)

from xgboost import XGBClassifier
model = XGBClassifier(n_estimators=150, max_depth=10, eval_metric='error', random_state=10)
model.fit(X_TRAIN, Y_TRAIN)

y_test_predicted = pd.DataFrame(model.predict(x_test)).rename(columns={0:'Survived'})
Y_TEST_PREDICTED = pd.DataFrame(model.predict(X_TEST))

from sklearn.metrics import roc_auc_score
print(roc_auc_score(Y_TEST, Y_TEST_PREDICTED))

# final = pd.concat([x_test_passenger_id, Y_TEST_PREDICTED], axis=1)
# final.to_csv('/12345.csv', index=False)

/Users/dabeen/opt/anaconda3/lib/python3.8/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Users/dabeen/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


0.8005013450721448


In [55]:
import pandas as pd
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import roc_auc_score
import warnings
warnings.filterwarnings('ignore')

x_train = pd.read_csv('/Users/dabeen/Desktop/DA_01/bigData-main/titanic_x_train.csv', encoding='cp949')
x_test = pd.read_csv('/Users/dabeen/Desktop/DA_01/bigData-main/titanic_x_test.csv', encoding='cp949')
y_train = pd.read_csv('/Users/dabeen/Desktop/DA_01/bigData-main/titanic_y_train.csv')

data = pd.concat([x_train, y_train], axis=1)

x_test_passenger_id = x_test['PassengerId']
x_train = x_train.drop(columns=['PassengerId'])
x_test = x_test.drop(columns=['PassengerId'])
y_train = y_train.drop(columns=['PassengerId'])
x_train = x_train.drop(columns=['티켓번호', '승객이름'])
x_test = x_test.drop(columns=['티켓번호', '승객이름'])
x_train = x_train.drop(columns=['나이', '객실번호'])
x_test = x_test.drop(columns=['나이', '객실번호'])

x_train['선착장'] = x_train['선착장'].fillna('S')
x_train['성별'] = x_train['성별'].replace('male', 0).replace('female', 1)
x_test['성별'] = x_test['성별'].replace('male', 0).replace('female', 1)

선착장_dummy = pd.get_dummies(x_train['선착장'], drop_first=True).rename(columns={'Q':'선착장Q', 'S':'선착장S'})
x_train = pd.concat([x_train, 선착장_dummy], axis=1)
x_train = x_train.drop(columns=['선착장'])
선착장_dummy = pd.get_dummies(x_test['선착장'], drop_first=True).rename(columns={'Q':'선착장Q', 'S':'선착장S'})
x_test = pd.concat([x_test, 선착장_dummy], axis=1)
x_test = x_test.drop(columns=['선착장'])

x_train['가족수'] = x_train['형제자매배우자수'] + x_train['부모자식수']
x_train = x_train.drop(columns=['형제자매배우자수', '부모자식수'])
x_test['가족수'] = x_test['형제자매배우자수'] + x_test['부모자식수']
x_test = x_test.drop(columns=['형제자매배우자수', '부모자식수'])

X_TRAIN, X_TEST, Y_TRAIN, Y_TEST = train_test_split(x_train, y_train, test_size=0.3, random_state=10)

# model = XGBClassifier()

# param_grid = {'n_estimators':[50, 100, 150, 200], 'max_depth':[6, 8, 10, 12, 14], 'min_samples_split':[0.2, 0.25, 0.3]}
# grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3)
# grid_search.fit(X_TRAIN, Y_TRAIN)

# print('best_score : ', grid_search.best_score_)
# print('best_params : ', grid_search.best_params_)

model = XGBClassifier(n_estimators=150, max_depth=10, eval_metric='error', random_state=10)
model.fit(X_TRAIN, Y_TRAIN)

y_test_predicted = pd.DataFrame(model.predict(x_test)).rename(columns={0:'Survived'})
Y_TEST_PREDICTED = pd.DataFrame(model.predict(X_TEST))

print('roc_auc_score : ', roc_auc_score(Y_TEST, Y_TEST_PREDICTED))

# final = pd.concat([x_test_passenger_id, Y_TEST_PREDICTED], axis=1)
# final.to_csv('/12345.csv', index=False)

roc_auc_score :  0.8005013450721448
